<a href="https://colab.research.google.com/github/kleberquaresma/alura_imersao_ia_google/blob/main/projeto_final_imersao_ia_alura_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

⚙

**Instalando a biblioteca do Google para uso da IA Generativa (Gemini):**

OBS.: *Para execução em um terminal local, fora do ambiente Colab, se faz necessário antes instalar o Python (https://www.python.org/downloads/) e certificar-se que o caminho da instalação dele está definido nas variáveis de ambiente do sistema operacional.*

In [3]:
!pip install -q -U google-generativeai


📥

**Importando as bibliotecas necessárias à execução do código.**

In [4]:
#-------------------------------------------------------------------------
# INÍCIO das Importações necessárias (bibliotecas, módulos, funções....)
#-------------------------------------------------------------------------

# Importação da biblioteca para melhor trabalhar com caminhos de arquivo.
import pathlib

# Importação da biblioteca para formatação de blocos de texto.
import textwrap

# Importação da biblioteca SDK para uso dos modelos de IA do Gemini.
import google.generativeai as genai

import pandas as pd

import numpy as np

# Importação de função que possibilita exibir resultados de forma mais amigável (tabelas, imagens, etc).
# Importação de classe que possibilita exibir textos utilizando a formação MarkDown.
from IPython.display import display, Markdown

# Importação de função que possibilita acessar dados, como a Chave da API, de forma segura.
from google.colab import userdata


#-------------------------------------------------------------------------
# FIM das Importações necessárias (bibliotecas, módulos, funções....)
#-------------------------------------------------------------------------

# Função que permitirá formatar o texto em MarkDown realizando algumas tratativas.
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

🔑

**Adicionando a chave da API que será utilizada para realização da integração com o ambiente do Gemini da Google:**

Obs.: *Necessário antes inserir a chave da API, previamente criada através do Google AI Studio, no gerenciador de chaves do Colab. Caso contrário inserir a chave em substituição ao texto 'GOOGLE_API_KEY'.*

In [6]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.

# genai.configure(api_key=os.environ["API_KEY"]) # para capturar a chave da API à partir da variável de ambiente (API_KEY) do SO.

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

♊

**Definindo as variáveis do Gemini:**

In [28]:
parametros_gemini = {
    "candidate_count": 1,
    "temperature": 0,
    "top_p": 0,
    "top_k": 0,
}

from google.generativeai.types import HarmCategory, HarmBlockThreshold

seguranca_gemini = {
    'HARASSMENT': 'block_none',
    'SEXUAL': 'block_none',
    'DANGEROUS': 'block_none',
    'HATE': 'block_none',
}

♊

**Definindo o modelo de IA do Gemini a ser utilizado:**

OBS.1: *Para interação multimodal (texto, imagens e vídeo) utilizar um dos modelos "VISIO".*

OBS.2: *Para lista de modelos de interação com conteúdo exclusivo utilizar "Embeddings" (embedContent)*

In [36]:
# Lista os modelos disponíveis:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

# Define o modelo a ser utilizado:
modelo = genai.GenerativeModel('gemini-pro',
                              safety_settings=seguranca_gemini,
                              generation_config=parametros_gemini)

modelo_escolhido = Markdown('**Modelo Escolhido:**')
display(modelo_escolhido, modelo.model_name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


**Modelo Escolhido:**

'models/gemini-pro'

Interação

In [33]:
df = pd.read_excel('guias.xlsx')
df

,TGS_NUMERO_GUIA,TGS_NUM_ATENDIMENTO,TGS_DATA_EMISSAO_GUIA,STATUS,TGS_DATA_CANCELAMENTO,TIPO_GUIA
0,67107,NaN,2017-08-21 16:10:52,Negado,NaT,Solicitação SP/SADT
1,67110,NaN,2017-08-21 17:08:38,Autorizado,NaT,Solicitação SP/SADT
2,67114,233926.0,2017-08-25 11:37:57,Aguardando Liberação para Pagamento,NaT,Faturamento SP/SADT
3,67180,NaN,2018-08-27 17:27:06,Autorizado,NaT,Solicitação SP/SADT
4,67181,NaN,2018-08-29 10:22:50,Autorizado,NaT,Solicitação SP/SADT
...,...,...,...,...,...,...
532,69932,NaN,2024-04-16 11:25:22,Autorizado,NaT,Solicitação SP/SADT
533,69933,NaN,45398,Encerrado Sem Pagamento,NaT,Honorario Individual
534,69935,NaN,45399,Encerrado Sem Pagamento,NaT,Honorario Individual
535,69970,NaN,2024-05-02 11:52:49,Autorizado,NaT,Solicitação SP/SADT


In [37]:
response = modelo.generate_content(f'{df}, dada essas informações, considerando o tipo da guia (TIPO_GUIA), há quantas guias de "Solicitação SP/SADT"?')

print(response.text)

367


In [44]:
def gerar_e_buscar_consulta(consulta, base, modelo):
    resultado_consulta = genai.embed_content(model="models/embedding-001",
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")

    #produtos_escalares = np.dot(np.stack(df["embeddings"]), embeddings_da_consulta["embedding"])

    #indice = np.argmax(produtos_escalares)
    return [resultado_consulta]

In [45]:
consulta = "Informe a quantidade de guias de solicitação de SP/SADT"

resposta = gerar_e_buscar_consulta(consulta, df, modelo)

print(resposta)

[{'embedding': [-0.021351064, -0.045297164, -0.0807509, -0.008271569, 0.08226806, -0.0004157302, 0.013902059, -0.07125409, 0.03576282, 0.07968104, 0.025289683, -0.019109644, -0.039427493, -0.016673272, -0.0045763445, -0.02689147, -0.020493766, -0.0030271318, -0.011982657, -0.027155386, 0.004136737, 0.047596585, -0.01901042, 0.009310005, -0.00067965244, -0.023681983, -0.009248729, -0.053934824, -0.027722187, 0.035891827, -0.052338615, 0.03218371, -0.053435493, 0.018168744, 0.0009159838, -0.04453855, 0.015771115, 0.03810764, -0.02879616, 0.033493247, 0.027494973, 0.02396882, 0.010838085, 0.0078230705, -0.007500075, -0.0033089493, 0.028314704, -0.0003393145, 0.010619205, -0.040835723, 0.00032103283, -0.00028141483, 0.09550287, 0.023857877, -0.0060027773, -0.06453616, 0.03902038, -0.0029246262, 0.0051320298, 0.029546518, 0.056064468, 0.014610022, -0.006913597, 0.034261074, -0.03185627, -0.048083007, 0.0015220117, -0.030470299, 0.05729053, -0.04147499, 0.01447766, -0.030922066, 0.0032363469